# AGEL Healthcare Data Pipeline – Diabetes 130 US Hospitals

This notebook shows an end-to-end data engineering pipeline built for the **AGEL BI Engineer** case study.

**Dataset:** Diabetes 130-US hospitals (1999–2008), UCI Machine Learning Repository  
**Goal:**  
- Ingest raw hospital encounters (CSV)  
- Run basic data quality checks  
- Transform data into a clean **silver** analytical table  
- Produce simple **healthcare metrics**: readmissions, length of stay, insulin therapy, demographics  




In [1]:
import pandas as pd
import numpy as np
import re
import json
import logging
from pathlib import Path
from io import StringIO

# ----------------------------------------------------------------------
# Logging config
# ----------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)


In [2]:
# ----------------------------------------------------------------------
# Paths & directories
# ----------------------------------------------------------------------

# In a notebook we use current folder as project root
PROJECT_ROOT = Path(".").resolve()
DATA_DIR = PROJECT_ROOT / "data"
RAW_DIR = DATA_DIR / "raw"
BRONZE_DIR = DATA_DIR / "bronze"
SILVER_DIR = DATA_DIR / "silver"
REPORT_DIR = PROJECT_ROOT / "reports"
DQ_REPORT_DIR = REPORT_DIR / "data_quality"

for d in [RAW_DIR, BRONZE_DIR, SILVER_DIR, REPORT_DIR, DQ_REPORT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# Change these if your filenames are different
MAIN_CSV_PATH = RAW_DIR / "diabetic_data.csv"
LOOKUP_CSV_PATH = RAW_DIR / "ids_mapping.csv"

MAIN_CSV_PATH, LOOKUP_CSV_PATH


(WindowsPath('C:/Users/sargsyantigran/Case_Studies/AGEL_test/notebooks/data/raw/diabetic_data.csv'),
 WindowsPath('C:/Users/sargsyantigran/Case_Studies/AGEL_test/notebooks/data/raw/ids_mapping.csv'))

## 1. Ingestion – Load Raw Data (Bronze)

In this section we:

- Safely load the main encounters CSV (`diabetic_data.csv`)
- Estimate how many rows were ingested vs total lines in the file
- Parse the lookup CSV (`ids_mapping.csv`) into three small tables  
  (admission type, discharge disposition, admission source)
- Save a **bronze** Parquet copy of the encounters
- Log a simple ingestion summary (JSON)


In [3]:
# ----------------------------------------------------------------------
# Ingestion helpers
# ----------------------------------------------------------------------

def count_data_lines(path, encoding="utf-8"):
    """Count number of *data* lines (excluding header)."""
    with open(path, "r", encoding=encoding) as f:
        return sum(1 for _ in f) - 1


def ingest_main_encounters(path, encoding="utf-8"):
    logging.info(f"Starting ingestion of main CSV from {path}")
    total_lines = count_data_lines(path, encoding=encoding)
    logging.info(f"Total lines in raw file (excluding header): {total_lines}")

    df = pd.read_csv(
        path,
        dtype=str,  # keep everything as string first
        na_values=["?", "NA", "NaN", "null"],
        on_bad_lines="skip",
        encoding=encoding,
    )

    rows_loaded, cols_loaded = df.shape
    rows_rejected = max(total_lines - rows_loaded, 0)

    ingestion_summary = {
        "file": str(path),
        "rows_loaded": int(rows_loaded),
        "columns_loaded": int(cols_loaded),
        "rows_in_file": int(total_lines),
        "rows_rejected_estimated": int(rows_rejected),
        "column_names": df.columns.tolist(),
    }

    logging.info(
        "Ingestion done: %s rows, %s columns, ~%s rejected rows.",
        rows_loaded,
        cols_loaded,
        rows_rejected,
    )

    return df, ingestion_summary


def parse_lookup_blocks(path, encoding="utf-8"):
    """
    Parse the IDs mapping file into three DataFrames:
    - admission_type_df
    - discharge_disposition_df
    - admission_source_df
    """
    with open(path, "r", encoding=encoding) as f:
        text = f.read()

    # Split on lines that are just "," or blank
    blocks = []
    current = []
    for line in text.splitlines():
        if line.strip() in {"", ","}:
            if current:
                blocks.append("\n".join(current))
                current = []
        else:
            current.append(line)
    if current:
        blocks.append("\n".join(current))

    if len(blocks) != 3:
        logging.warning("Expected 3 blocks in lookup file, found %s", len(blocks))

    dfs = []
    for block in blocks:
        df_block = pd.read_csv(StringIO(block), dtype=str)
        dfs.append(df_block)

    admission_type_df, discharge_disp_df, admission_source_df = dfs

    # Just in case: use consistent column names
    admission_type_df = admission_type_df.rename(
        columns={"admission_type_id": "admission_type_id"}
    )
    discharge_disp_df = discharge_disp_df.rename(
        columns={"discharge_disposition_id": "discharge_disposition_id"}
    )
    admission_source_df = admission_source_df.rename(
        columns={"admission_source_id": "admission_source_id"}
    )

    return admission_type_df, discharge_disp_df, admission_source_df


In [4]:
# Ingest the main encounters data
df_raw, ingestion_report = ingest_main_encounters(MAIN_CSV_PATH)

# Save bronze Parquet
bronze_main_path = BRONZE_DIR / "diabetic_encounters_bronze.parquet"
df_raw.to_parquet(bronze_main_path, index=False)

# Save ingestion report as JSON
ingestion_report_path = DQ_REPORT_DIR / "ingestion_report.json"
with open(ingestion_report_path, "w") as f:
    json.dump(ingestion_report, f, indent=2)

df_raw.head(), ingestion_report


2025-12-02 12:23:12,952 [INFO] Starting ingestion of main CSV from C:\Users\sargsyantigran\Case_Studies\AGEL_test\notebooks\data\raw\diabetic_data.csv
2025-12-02 12:23:13,061 [INFO] Total lines in raw file (excluding header): 101766
2025-12-02 12:23:13,411 [INFO] Ingestion done: 101766 rows, 50 columns, ~0 rejected rows.


(  encounter_id patient_nbr             race  gender      age weight  \
 0      2278392     8222157        Caucasian  Female   [0-10)    NaN   
 1       149190    55629189        Caucasian  Female  [10-20)    NaN   
 2        64410    86047875  AfricanAmerican  Female  [20-30)    NaN   
 3       500364    82442376        Caucasian    Male  [30-40)    NaN   
 4        16680    42519267        Caucasian    Male  [40-50)    NaN   
 
   admission_type_id discharge_disposition_id admission_source_id  \
 0                 6                       25                   1   
 1                 1                        1                   7   
 2                 1                        1                   7   
 3                 1                        1                   7   
 4                 1                        1                   7   
 
   time_in_hospital  ... citoglipton insulin glyburide-metformin  \
 0                1  ...          No      No                  No   
 1            

## 2. Data Validation & Data Quality

Here we build a small validation layer that:

- Counts missing values per column  
- Looks for duplicates (overall and by `encounter_id`)  
- Checks basic logical constraints:
  - Age brackets within 0–120 years
  - `time_in_hospital` between 1 and 14 days
  - Valid gender codes (`Male`, `Female`, `Unknown/Invalid`)

The results are saved as a JSON data quality report.


In [5]:
# ----------------------------------------------------------------------
# Validation
# ----------------------------------------------------------------------

def validate_missing_and_duplicates(df):
    n_rows, n_cols = df.shape

    missing = df.isna().sum().reset_index()
    missing.columns = ["column", "missing_count"]
    missing["missing_pct"] = missing["missing_count"] / n_rows

    dup_all_rows = int(df.duplicated().sum())
    if "encounter_id" in df.columns:
        dup_encounters = int(df.duplicated(subset=["encounter_id"]).sum())
    else:
        dup_encounters = None

    return {
        "row_count": int(n_rows),
        "column_count": int(n_cols),
        "missing_by_column": missing.to_dict(orient="records"),
        "duplicates_all_rows": dup_all_rows,
        "duplicates_by_encounter_id": dup_encounters,
    }


def parse_age_range(age_str):
    """Convert age like '[60-70)' into (60, 70), or (None, None) if invalid."""
    if pd.isna(age_str):
        return (None, None)
    age_str = str(age_str).strip()
    if not (age_str.startswith("[") and age_str.endswith(")") and "-" in age_str):
        return (None, None)
    inner = age_str[1:-1]
    parts = inner.split("-")
    if len(parts) != 2:
        return (None, None)
    try:
        low = int(parts[0])
        high = int(parts[1])
        return (low, high)
    except ValueError:
        return (None, None)


def validate_logical_constraints(df):
    issues = {}

    # --- age check ---
    if "age" in df.columns:
        age_ranges = df["age"].dropna().apply(parse_age_range)
        lows = [x[0] for x in age_ranges if x[0] is not None]
        highs = [x[1] for x in age_ranges if x[1] is not None]

        invalid_age_rows = df[
            df["age"].notna()
            & df["age"].apply(lambda x: parse_age_range(x)[0] is None)
        ].shape[0]

        issues["age_min_observed"] = min(lows) if lows else None
        issues["age_max_observed"] = max(highs) if highs else None
        issues["age_invalid_rows"] = int(invalid_age_rows)

        issues["age_out_of_0_120"] = int(
            sum((low < 0 or high > 120) for low, high in zip(lows, highs))
        )

    # --- time_in_hospital check ---
    if "time_in_hospital" in df.columns:
        tih = pd.to_numeric(df["time_in_hospital"], errors="coerce")
        invalid_tih = tih.isna().sum()
        too_low = (tih < 1).sum()
        too_high = (tih > 14).sum()

        issues["time_in_hospital_invalid_rows"] = int(invalid_tih)
        issues["time_in_hospital_less_than_1"] = int(too_low)
        issues["time_in_hospital_greater_than_14"] = int(too_high)

    # --- gender check ---
    if "gender" in df.columns:
        valid_gender = {"Male", "Female", "Unknown/Invalid"}
        unique_gender = set(df["gender"].dropna().unique())
        invalid_gender_values = sorted(list(unique_gender - valid_gender))
        issues["gender_unique_values"] = sorted(list(unique_gender))
        issues["gender_invalid_values"] = invalid_gender_values

    return issues


def run_validation(df, report_dir, report_name="data_quality_report.json"):
    logging.info("Starting data validation...")
    report = {}
    report["missing_and_duplicates"] = validate_missing_and_duplicates(df)
    report["logical_checks"] = validate_logical_constraints(df)

    report_path = report_dir / report_name
    with open(report_path, "w") as f:
        json.dump(report, f, indent=2)

    logging.info("Data quality report written to %s", report_path)
    return report


In [6]:
#run validation

dq_report = run_validation(df_raw, DQ_REPORT_DIR)
dq_report



2025-12-02 12:24:32,656 [INFO] Starting data validation...
2025-12-02 12:24:33,308 [INFO] Data quality report written to C:\Users\sargsyantigran\Case_Studies\AGEL_test\notebooks\reports\data_quality\data_quality_report.json


{'missing_and_duplicates': {'row_count': 101766,
  'column_count': 50,
  'missing_by_column': [{'column': 'encounter_id',
    'missing_count': 0,
    'missing_pct': 0.0},
   {'column': 'patient_nbr', 'missing_count': 0, 'missing_pct': 0.0},
   {'column': 'race',
    'missing_count': 2273,
    'missing_pct': 0.022335554114340742},
   {'column': 'gender', 'missing_count': 0, 'missing_pct': 0.0},
   {'column': 'age', 'missing_count': 0, 'missing_pct': 0.0},
   {'column': 'weight',
    'missing_count': 98569,
    'missing_pct': 0.9685847925633315},
   {'column': 'admission_type_id', 'missing_count': 0, 'missing_pct': 0.0},
   {'column': 'discharge_disposition_id',
    'missing_count': 0,
    'missing_pct': 0.0},
   {'column': 'admission_source_id', 'missing_count': 0, 'missing_pct': 0.0},
   {'column': 'time_in_hospital', 'missing_count': 0, 'missing_pct': 0.0},
   {'column': 'payer_code',
    'missing_count': 40256,
    'missing_pct': 0.395574160328597},
   {'column': 'medical_specialty',

## 3. Transform Helpers – Diagnosis, Medications, Encodings

Now we define helper functions used in the **silver** transformation:

- Clean diagnosis codes (`diag_1`, `diag_2`, `diag_3`) and group `diag_1` into categories  
- Standardize medication status (No / Steady / Up / Down)  
- Encode gender, race and readmission flags to more convenient formats


In [11]:
# ----------------------------------------------------------------------
# Transform helpers
# ----------------------------------------------------------------------

def clean_diag_code(series):
    """Standardize diagnosis codes."""
    s = series.astype("string").str.strip()
    s = s.replace({"?": pd.NA, "": pd.NA})
    s = s.str.upper()
    return s


def standardize_med_status(series):
    """Standardize medication columns."""
    mapping = {
        "No": "no",
        "Steady": "steady",
        "Up": "increased",
        "Down": "decreased",
        "?": pd.NA,
        "NA": pd.NA,
        "None": pd.NA,
    }
    s = series.astype("string").str.strip()
    return s.map(mapping).astype("string")


def encode_gender(df):
    mapping = {
        "Male": "M",
        "Female": "F",
        "Unknown/Invalid": "U",
    }
    df["gender_clean"] = df["gender"].map(mapping).astype("string")

    df["gender_female_flag"] = (
        df["gender_clean"].map({"F": 1, "M": 0, "U": pd.NA})
    ).astype("Int64")
    return df


def encode_race(df):
    race_map = {
        "Caucasian": "Caucasian",
        "AfricanAmerican": "African American",
        "Asian": "Asian",
        "Hispanic": "Hispanic",
        "Other": "Other",
        "?": pd.NA,
        "": pd.NA,
    }
    df["race_clean"] = (
        df["race"].astype("string").str.strip().map(race_map)
    ).astype("string")
    return df


def encode_readmitted(df):
    # clean original values to a canonical form
    s = df["readmitted"].astype("string").str.strip().str.upper()
    df["readmitted_raw_clean"] = s

    # mapping to numeric flags
    mapping_any = {"NO": 0, "<30": 1, ">30": 1}   # any readmission
    mapping_30d = {"NO": 0, "<30": 1, ">30": 0}   # only <30 days

    df["readmitted_any_flag"] = s.map(mapping_any).astype("Int64")
    df["readmitted_30d_flag"] = s.map(mapping_30d).astype("Int64")

    return df



## 4. Transform to Silver Table

Here we create the main transformation function:

- Cast numeric and ID columns
- Clean diagnosis codes and build `diag_1_group`
- Clean medication columns and compute number of active diabetes meds
- Encode gender, race, readmission
- Clean lab results (A1C, max_glu_serum)
- Join lookup descriptions (admission type, discharge disposition, admission source)
- Normalize column names to **snake_case**


In [8]:
# ----------------------------------------------------------------------
# Transform main
# ----------------------------------------------------------------------

def transform_encounters(df_raw, admission_type_df, discharge_disp_df, admission_source_df):
    df = df_raw.copy()

    # 1) Basic numeric types
    numeric_cols = [
        "time_in_hospital",
        "num_lab_procedures",
        "num_procedures",
        "num_medications",
        "number_outpatient",
        "number_emergency",
        "number_inpatient",
        "number_diagnoses",
    ]
    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")

    # IDs as string
    id_cols = [
        "encounter_id",
        "patient_nbr",
        "admission_type_id",
        "discharge_disposition_id",
        "admission_source_id",
    ]
    for col in id_cols:
        if col in df.columns:
            df[col] = df[col].astype("string")

    # 2) Diagnosis codes
    diag_cols = ["diag_1", "diag_2", "diag_3"]
    for col in diag_cols:
        if col in df.columns:
            df[col + "_clean"] = clean_diag_code(df[col])

    def diag_group(code):
        if code is None or pd.isna(code):
            return None
        code = str(code)
        # diabetes explicitly (ICD-9 250.xx)
        if code.startswith("250"):
            return "diabetes"
        try:
            match = re.match(r"(\d+(\.\d+)?)", code)
            if not match:
                return "other"
            num = float(match.group(1))
        except Exception:
            return "other"
        if 390 <= num <= 459:
            return "circulatory"
        if 460 <= num <= 519:
            return "respiratory"
        if 520 <= num <= 579:
            return "digestive"
        return "other"

    df["diag_1_group"] = df["diag_1_clean"].apply(diag_group).astype("string")

    # 3) Medications
    med_cols = [
        "metformin", "repaglinide", "nateglinide", "chlorpropamide",
        "glimepiride", "acetohexamide", "glipizide", "glyburide",
        "tolbutamide", "pioglitazone", "rosiglitazone", "acarbose",
        "miglitol", "troglitazone", "tolazamide", "examide",
        "citoglipton", "insulin", "glyburide-metformin",
        "glipizide-metformin", "glimepiride-pioglitazone",
        "metformin-rosiglitazone", "metformin-pioglitazone",
    ]

    active_flags = []
    for col in med_cols:
        if col in df.columns:
            clean_col = col + "_clean"
            df[clean_col] = standardize_med_status(df[col])
            flag_col = clean_col + "_active_flag"
            df[flag_col] = df[clean_col].map(
                {"steady": 1, "increased": 1, "decreased": 1, "no": 0}
            ).astype("Int64")
            active_flags.append(flag_col)

    if active_flags:
        df["num_active_diabetes_meds"] = df[active_flags].sum(axis=1).astype("Int64")

    # 4) Encode key attributes
    df = encode_gender(df)
    df = encode_race(df)
    df = encode_readmitted(df)

    # Labs
    lab_cols = ["A1Cresult", "max_glu_serum"]
    for col in lab_cols:
        if col in df.columns:
            s = df[col].astype("string").str.strip()
            s = s.replace({"None": pd.NA, "?": pd.NA, "": pd.NA})
            df[col + "_clean"] = s

    # 5) Join lookup tables
    admission_type_df = admission_type_df.copy()
    discharge_disp_df = discharge_disp_df.copy()
    admission_source_df = admission_source_df.copy()

    admission_type_df["admission_type_id"] = admission_type_df["admission_type_id"].astype("string")
    discharge_disp_df["discharge_disposition_id"] = discharge_disp_df["discharge_disposition_id"].astype("string")
    admission_source_df["admission_source_id"] = admission_source_df["admission_source_id"].astype("string")

    if "description" in admission_type_df.columns:
        admission_type_df = admission_type_df.rename(columns={"description": "admission_type_desc"})
    if "description" in discharge_disp_df.columns:
        discharge_disp_df = discharge_disp_df.rename(columns={"description": "discharge_disposition_desc"})
    if "description" in admission_source_df.columns:
        admission_source_df = admission_source_df.rename(columns={"description": "admission_source_desc"})

    df = df.merge(admission_type_df, on="admission_type_id", how="left")
    df = df.merge(discharge_disp_df, on="discharge_disposition_id", how="left")
    df = df.merge(admission_source_df, on="admission_source_id", how="left")

    # 6) Column naming consistency
    def to_snake(name):
        name = name.replace(" ", "_").replace("-", "_").replace("/", "_")
        return name.lower()

    df.columns = [to_snake(c) for c in df.columns]

    return df


## 5. Healthcare Summary Metrics

From the silver table we create several simple but meaningful metrics:

- Overall cohort summary (encounters, patients, LOS, readmission rates)
- Readmission rate by age band
- Readmission rate by insulin status
- Race × gender summary (encounters, LOS, readmission)


In [9]:
# ----------------------------------------------------------------------
# Summary metrics
# ----------------------------------------------------------------------

def generate_summaries(df_silver, report_dir):
    # Overall summary
    df = df_silver.copy()
    df["time_in_hospital_num"] = pd.to_numeric(df["time_in_hospital"], errors="coerce")
    df["num_medications_num"] = pd.to_numeric(df["num_medications"], errors="coerce")
    readmitted_upper = df["readmitted"].str.upper().str.strip()

    summary_overall = pd.DataFrame([{
        "n_encounters": len(df),
        "n_unique_patients": df["patient_nbr"].nunique(),
        "mean_length_of_stay_days": df["time_in_hospital_num"].mean(),
        "median_length_of_stay_days": df["time_in_hospital_num"].median(),
        "mean_num_medications": df["num_medications_num"].mean(),
        "readmission_rate_any": (readmitted_upper != "NO").mean(),
        "readmission_rate_30d": (readmitted_upper == "<30").mean(),
    }])
    summary_overall.to_csv(report_dir / "summary_overall_metrics.csv", index=False)

    # Readmission by age
    df_age = df_silver.copy()
    df_age["readmitted_any"] = df_age["readmitted"].str.upper().str.strip().ne("NO")
    readmission_by_age = (
        df_age.groupby("age", dropna=False)["readmitted_any"]
        .agg(["count", "mean"])
        .reset_index()
        .rename(columns={"count": "n_encounters", "mean": "readmission_rate"})
        .sort_values("age")
    )
    readmission_by_age.to_csv(report_dir / "readmission_by_age.csv", index=False)

    # Readmission by insulin
    df_ins = df_silver.copy()
    df_ins["readmitted_any"] = df_ins["readmitted"].str.upper().str.strip().ne("NO")
    insulin_readmission = (
        df_ins.groupby("insulin", dropna=False)["readmitted_any"]
        .agg(["count", "mean"])
        .reset_index()
        .rename(columns={"count": "n_encounters", "mean": "readmission_rate"})
        .sort_values("insulin")
    )
    insulin_readmission.to_csv(report_dir / "readmission_by_insulin.csv", index=False)

    # Race & gender
    df_rg = df_silver.copy()
    df_rg["time_in_hospital_num"] = pd.to_numeric(df_rg["time_in_hospital"], errors="coerce")
    df_rg["readmitted_any"] = df_rg["readmitted"].str.upper().str.strip().ne("NO")
    race_gender_summary = (
        df_rg.groupby(["race", "gender"], dropna=False)
        .agg(
            n_encounters=("encounter_id", "count"),
            mean_los_days=("time_in_hospital_num", "mean"),
            readmission_rate=("readmitted_any", "mean"),
        )
        .reset_index()
        .sort_values(["race", "gender"])
    )
    race_gender_summary.to_csv(report_dir / "race_gender_summary.csv", index=False)


## 6. Run the Full Pipeline (Notebook Version)

Now we:

1. Re-use the **raw DataFrame** from ingestion  
2. Parse the lookup tables  
3. Transform to **silver**  
4. Save silver outputs  
5. Generate healthcare summaries  

This mirrors what the standalone `pipeline.py` script does, but step-by-step.


In [12]:
# 1) We already have df_raw from ingestion step above

# 2) Parse lookup tables
admission_type_df, discharge_disp_df, admission_source_df = parse_lookup_blocks(LOOKUP_CSV_PATH)

# 3) Transform to silver
df_silver = transform_encounters(df_raw, admission_type_df, discharge_disp_df, admission_source_df)

# 4) Save silver outputs
silver_parquet_path = SILVER_DIR / "diabetic_encounters_silver.parquet"
silver_csv_path = SILVER_DIR / "diabetic_encounters_silver.csv"

df_silver.to_parquet(silver_parquet_path, index=False)
df_silver.to_csv(silver_csv_path, index=False)

# 5) Generate summaries
generate_summaries(df_silver, REPORT_DIR)

print("Rows in silver:", df_silver.shape[0])
print("Columns in silver:", df_silver.shape[1])
silver_parquet_path, silver_csv_path


Rows in silver: 101766
Columns in silver: 112


(WindowsPath('C:/Users/sargsyantigran/Case_Studies/AGEL_test/notebooks/data/silver/diabetic_encounters_silver.parquet'),
 WindowsPath('C:/Users/sargsyantigran/Case_Studies/AGEL_test/notebooks/data/silver/diabetic_encounters_silver.csv'))

## 7. Quick Exploration of Silver Table

Here we quickly look at the transformed dataset and some summary tables.


In [13]:
df_silver.head()


,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,gender_female_flag,race_clean,readmitted_raw_clean,readmitted_any_flag,readmitted_30d_flag,a1cresult_clean,max_glu_serum_clean,admission_type_desc,discharge_disposition_desc,admission_source_desc
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,1,Caucasian,NO,0,0,<NA>,<NA>,NaN,Not Mapped,Physician Referral
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,1,Caucasian,>30,1,0,<NA>,<NA>,Emergency,Discharged to home,Emergency Room
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,1,African American,NO,0,0,<NA>,<NA>,Emergency,Discharged to home,Emergency Room
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,0,Caucasian,NO,0,0,<NA>,<NA>,Emergency,Discharged to home,Emergency Room
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,0,Caucasian,NO,0,0,<NA>,<NA>,Emergency,Discharged to home,Emergency Room


In [14]:
pd.read_csv(REPORT_DIR / "summary_overall_metrics.csv")


,n_encounters,n_unique_patients,mean_length_of_stay_days,median_length_of_stay_days,mean_num_medications,readmission_rate_any,readmission_rate_30d
0,101766,71518,4.395987,4.0,16.021844,0.460881,0.111599


In [15]:
pd.read_csv(REPORT_DIR / "readmission_by_age.csv").head()


,age,n_encounters,readmission_rate
0,[0-10),161,0.180124
1,[10-20),691,0.382055
2,[20-30),1657,0.450211
3,[30-40),3775,0.426755
4,[40-50),9685,0.444502


In [16]:
pd.read_csv(REPORT_DIR / "readmission_by_insulin.csv")


,insulin,n_encounters,readmission_rate
0,Down,12218,0.527910
1,No,47383,0.436971
2,Steady,30849,0.451068
3,Up,11316,0.515376


In [17]:
df = df_silver.copy()
df["time_in_hospital_num"] = pd.to_numeric(df["time_in_hospital"], errors="coerce")
df["readmitted_any"] = df["readmitted"].str.upper().str.strip().ne("NO")

race_gender_summary = (
    df.groupby(["race", "gender"], dropna=False)
      .agg(
          n_encounters=("encounter_id", "count"),
          mean_los_days=("time_in_hospital_num", "mean"),
          readmission_rate=("readmitted_any", "mean"),
      )
      .reset_index()
      .sort_values(["race", "gender"])
)

race_gender_summary.to_csv(REPORT_DIR / "race_gender_summary.csv", index=False)
race_gender_summary

,race,gender,n_encounters,mean_los_days,readmission_rate
0,AfricanAmerican,Female,11728,4.541269,0.460607
1,AfricanAmerican,Male,7482,4.455493,0.452686
2,Asian,Female,318,3.899371,0.355346
3,Asian,Male,323,4.089783,0.349845
4,Caucasian,Female,39689,4.481443,0.479931
5,Caucasian,Male,36410,4.281379,0.457786
6,Hispanic,Female,1092,4.086081,0.415751
7,Hispanic,Male,945,4.02963,0.423280
8,Other,Female,748,4.252674,0.402406
9,Other,Male,757,4.298547,0.383091
